In [1]:
from ortools.linear_solver import pywraplp
from dataclasses import dataclass
import pandas as pd   
from datetime import datetime
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import threading
import random
import concurrent

## Load Pods


In [2]:
pods_data_src = pd.read_csv("DEV_10_dec - pods.csv")

pods_data_src['req_cpu_milli_core'] = pd.to_numeric(pods_data_src['req_cpu_milli_core'])

pods_data_src['req_mem_mb'] = pd.to_numeric(pods_data_src['req_mem_byte']) / 1000000


pods_data = pods_data_src[(pods_data_src.node_group == 3) &  
                          (pods_data_src.owner_kind != 'DaemonSet') & 
                          (pods_data_src.req_cpu_milli_core>0) &
                          (pods_data_src.req_mem_byte>0) ][['namespace', 'pod_name', 'req_mem_mb', 'req_cpu_milli_core' ]]

daemonset = pods_data_src[(pods_data_src.node_group == 3) & (pods_data_src.owner_kind == 'DaemonSet')].\
            groupby("owner_name").agg({'req_cpu_milli_core':'mean', 'req_mem_byte':'mean'})

overhead = {'cpu': daemonset.req_cpu_milli_core.sum(), 'memory': daemonset.req_mem_byte.sum()/1000000 }

## Load Nodes

In [3]:
nodes_data = pd.read_csv("instances.csv")

nodes_data['cpu']  = pd.to_numeric(nodes_data.vCPUs, errors='coerce')*1000

nodes_data['memory'] = pd.to_numeric(nodes_data.Memory, errors='coerce')*1000

nodes_data['cost'] =  pd.to_numeric(nodes_data['Linux Reserved cost'], errors='coerce')

nodes_data = nodes_data[['API Name','Name','cpu', 'memory', 'cost']]

total_node_types = len(nodes_data)

nodes_data = nodes_data[(nodes_data.cpu >= pods_data.req_cpu_milli_core.max()) & (nodes_data.memory >= pods_data.req_mem_mb.max()) ]

suitable_node_types = len(nodes_data)

nodes_data.cpu = nodes_data.cpu - overhead['cpu']
nodes_data.memory = nodes_data.cpu - overhead['memory']

print(f"Detected {suitable_node_types} suitable nodes out of {total_node_types} total")


Detected 302 suitable nodes out of 382 total


In [4]:
#Create a data model
#
def create_data_model(cpu, memory, pods):
    """Create the data for the example."""
    data = {}    
    data['req_cpu'] = pods['req_cpu_milli_core'].tolist()
    data['req_memory'] = pods['req_mem_mb'].tolist()
    data['items'] = list(range(len( pods)))
    data['bins'] = data['items']
    data['cpu_capacity'] = cpu
    data['memory_capacity'] = memory
    return data

In [5]:
def create_solver(data):
    x = {}
    y = {}
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #solver.SetNumThreads(6)
    
   # solver.SetTimeLimit(30000)

    # Variables
    # x[i, j] = 1 if item i is packed in bin j.

    for i in data['items']:
        for j in data['bins']:
            x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

    # y[j] = 1 if bin j is used.

    for j in data['bins']:
        y[j] = solver.IntVar(0, 1, 'y[%i]' % j)
    
  
    # Constraints
    # Each item must be in exactly one bin.
    for i in data['items']:
        solver.Add(sum(x[i, j] for j in data['bins']) == 1)

    # The amount packed in each bin cannot exceed its cpu capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['req_cpu'][i] for i in data['items']) <= y[j] *
            data['cpu_capacity'])


    # The amount packed in each bin cannot exceed its capacity.
    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['req_cpu'][i] for i in data['items']) <= y[j] * data['cpu_capacity'])


    for j in data['bins']:
        solver.Add(
            sum(x[(i, j)] * data['req_memory'][i] for i in data['items']) <= y[j] * data['memory_capacity'])
    
    
    return solver, x, y

In [8]:
def solve(solver, data, x, y, pods):
    solver.Minimize(solver.Sum([y[j] for j in data['bins']]))
    status = solver.Solve()
    solution = []
    if status == pywraplp.Solver.OPTIMAL:
        num_bins = 0.
        for j in data['bins']:
            if y[j].solution_value() == 1:
                bin_items = []
                bin_pods = pd.DataFrame(columns = pods.columns, index = pods.index).sort_index().truncate(-1, -1 ).reindex()
                bin_cpu = 0
                bin_memory = 0
                for i in data['items']:
                    if x[i, j].solution_value() > 0:
                        bin_items.append(i)
                       # print(pods.iloc[[i]])
                        bin_pods = bin_pods.append(pods.iloc[[i]])
                        bin_cpu += data['req_cpu'][i]
                        bin_memory += data['req_memory'][i]
                if bin_cpu > 0 or bin_memory>0:
                    solution.append(bin_pods.copy())
                    num_bins += 1
                    #print('Bin number', j)
                    #print('  Items packed:', bin_items)
                    #print('  Pods packed:', bin_pods)
                    #print(f"  Total cpu: {bin_cpu}, cpu utilization: {round(bin_cpu / data['cpu_capacity'] * 100, 2)}%")
                    #print(f"  Total memory: {bin_memory}, memory utilization {round(bin_memory / data['memory_capacity'] * 100, 2)}%")
                    #print()
        #print()        
        #print('Number of bins used:', num_bins)
        #print('Time = ', solver.WallTime()/1000/60, ' mins')
    else:
        print('The problem does not have an optimal solution.')
        return []
    return solution

In [9]:
def get_solution(curr_node, pods_data):
    global solutions
    print (f"{datetime.now().strftime('%D %H:%M:%S')}: Solving for {curr_node['API Name']} (cpu: {curr_node.cpu}, memory: {curr_node.memory})")
    
    if (len(solutions[(solutions.cpu == curr_node.cpu) & (solutions.memory == curr_node.memory)]) == 0):
        
        data = create_data_model(curr_node.cpu, curr_node.memory, pods_data)
        solver,x,y = create_solver(data)
        solution = solve(solver, data, x, y, pods_data)
        
        if len(solution) > 0:
            print (f"{datetime.now().strftime('%D %H:%M:%S')}: Solution for {curr_node['API Name']}: {len(solution)} nodes, cost: {len(solution) * curr_node.cost}")
        else:
            print (f"{datetime.now().strftime('%D %H:%M:%S')}: No solution found for {curr_node['API Name']}: {len(solution)} nodes, cost: {len(solution) * curr_node.cost}")
    else:
        old_node = solutions[(solutions.cpu == curr_node.cpu) & (solutions.memory == curr_node.memory)].iloc[0]
        solution = old_node.solution
        print (f"{datetime.now().strftime('%D %H:%M:%S')}: Reusing solution for {old_node['name']}")
            
    
    
    #solutions = solutions.append({'name': curr_node['API Name'], 'cpu': curr_node['cpu'], 'memory': curr_node['memory'], 
    #                 'num_nodes': len(solution), 'cost': curr_cost, 
    #                 'solution': solution}, ignore_index = True)
    
    #if curr_cost < min_cost:
    #    best_node = nodes_data.iloc[i]
    #    min_cost = curr_cost
    #    best_solution = solution
    
    
    
    return pd.DataFrame(
        np.array([curr_node['API Name'], curr_node['cpu'], curr_node['memory'],
                len(solution), curr_node.cost * len(solution), solution]).reshape(1,6),
        columns = solutions.columns)

In [10]:
from multiprocessing.pool import ThreadPool

solutions = pd.DataFrame(columns = ["name", "cpu", "memory", "num_nodes", "cost", "solution"])


min_cost = float('inf')
best_solution = []
best_node = None

pods = pods_data[:900]

print(f"Starting... {datetime.now().strftime('%D %H:%M:%S')}")

executor = ThreadPoolExecutor(max_workers=12)
futures = []

for i in range(1,  len(nodes_data)):
    curr_node = nodes_data.iloc[i]    
    solution = get_solution(curr_node, pods)
    solutions = solutions.append(solution, ignore_index = True)

    #futures.append(executor.submit(solve_wrapper, curr_node, pods_data))
    
#for future in concurrent.futures.as_completed(futures):
        
#        to_append = pd.DataFrame(np.array(future.result()).reshape(1,6), columns = solutions.columns)
        #print(to_append.columns)
#        solutions = solutions.append(to_append, ignore_index = True)
        #print (future.result())
        
print("All solutions found")

#20 nodes - 1 hour with 6 threads
#20 nodes - 1 hour with 3 threads
#20 nodes - 1 hour with 12 threads


#Number of Pods
#200 pods  7 sec
#400 pods - 64 sec
#600 pods - 9 mins
#800 pods - 36 mins
#1000

#2000 pods - stopped after 18 hours?


#on parallelism https://github.com/google/or-tools/issues/1656

Starting... 12/24/20 00:38:25
12/24/20 00:38:25: Solving for m5a.2xlarge (cpu: 7628.0, memory: 7021.923072)
The problem does not have an optimal solution.
12/24/20 00:38:25: No solution found for m5a.2xlarge: 0 nodes, cost: 0.0
12/24/20 00:38:26: Solving for r5n.12xlarge (cpu: 47628.0, memory: 47021.923072)


<ipython-input-9-01ee4b658818>:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array([curr_node['API Name'], curr_node['cpu'], curr_node['memory'],


12/24/20 00:38:28: Solution for r5n.12xlarge: 4 nodes, cost: 9.012
12/24/20 00:38:28: Solving for r5ad.xlarge (cpu: 3628.0, memory: 3021.923072)
The problem does not have an optimal solution.
12/24/20 00:38:29: No solution found for r5ad.xlarge: 0 nodes, cost: 0.0
12/24/20 00:38:29: Solving for r5n.xlarge (cpu: 3628.0, memory: 3021.923072)
12/24/20 00:38:29: Reusing solution for r5ad.xlarge
12/24/20 00:38:29: Solving for r5dn.xlarge (cpu: 3628.0, memory: 3021.923072)
12/24/20 00:38:29: Reusing solution for r5ad.xlarge
12/24/20 00:38:29: Solving for i2.xlarge (cpu: 3628.0, memory: 3021.923072)
12/24/20 00:38:29: Reusing solution for r5ad.xlarge
12/24/20 00:38:29: Solving for m5n.16xlarge (cpu: 63628.0, memory: 63021.923072)
12/24/20 00:38:32: Solution for m5n.16xlarge: 3 nodes, cost: 7.197
12/24/20 00:38:32: Solving for d2.8xlarge (cpu: 35628.0, memory: 35021.923072)
12/24/20 00:38:36: Solution for d2.8xlarge: 5 nodes, cost: 16.080000000000002
12/24/20 00:38:36: Solving for inf1.xlarge 

12/24/20 00:38:54: Solution for inf1.6xlarge: 8 nodes, cost: 9.592
12/24/20 00:38:54: Solving for r6gd.2xlarge (cpu: 7628.0, memory: 7021.923072)
12/24/20 00:38:54: Reusing solution for m5a.2xlarge
12/24/20 00:38:54: Solving for c5a.xlarge (cpu: 3628.0, memory: 3021.923072)
12/24/20 00:38:54: Reusing solution for r5ad.xlarge
12/24/20 00:38:54: Solving for m5n.24xlarge (cpu: 95628.0, memory: 95021.923072)
12/24/20 00:38:54: Reusing solution for r5n.24xlarge
12/24/20 00:38:54: Solving for m4.xlarge (cpu: 3628.0, memory: 3021.923072)
12/24/20 00:38:54: Reusing solution for r5ad.xlarge
12/24/20 00:38:54: Solving for r5a.4xlarge (cpu: 15628.0, memory: 15021.923072)
12/24/20 00:38:54: Reusing solution for a1.metal
12/24/20 00:38:54: Solving for r5b.metal (cpu: 95628.0, memory: 95021.923072)
12/24/20 00:38:54: Reusing solution for r5n.24xlarge
12/24/20 00:38:54: Solving for m4.16xlarge (cpu: 63628.0, memory: 63021.923072)
12/24/20 00:38:54: Reusing solution for m5n.16xlarge
12/24/20 00:38:54:

12/24/20 00:38:57: Solving for r5b.24xlarge (cpu: 95628.0, memory: 95021.923072)
12/24/20 00:38:57: Reusing solution for r5n.24xlarge
12/24/20 00:38:57: Solving for i3.8xlarge (cpu: 31628.0, memory: 31021.923072)
12/24/20 00:38:57: Reusing solution for i2.8xlarge
12/24/20 00:38:57: Solving for cr1.8xlarge (cpu: 31628.0, memory: 31021.923072)
12/24/20 00:38:57: Reusing solution for i2.8xlarge
12/24/20 00:38:57: Solving for c6g.12xlarge (cpu: 47628.0, memory: 47021.923072)
12/24/20 00:38:57: Reusing solution for r5n.12xlarge
12/24/20 00:38:57: Solving for m5n.8xlarge (cpu: 31628.0, memory: 31021.923072)
12/24/20 00:38:57: Reusing solution for i2.8xlarge
12/24/20 00:38:57: Solving for m6gd.8xlarge (cpu: 31628.0, memory: 31021.923072)
12/24/20 00:38:57: Reusing solution for i2.8xlarge
12/24/20 00:38:57: Solving for m5a.24xlarge (cpu: 95628.0, memory: 95021.923072)
12/24/20 00:38:57: Reusing solution for r5n.24xlarge
12/24/20 00:38:57: Solving for r6g.4xlarge (cpu: 15628.0, memory: 15021.92

12/24/20 00:39:02: Reusing solution for a1.metal
12/24/20 00:39:02: Solving for c5.12xlarge (cpu: 47628.0, memory: 47021.923072)
12/24/20 00:39:02: Reusing solution for r5n.12xlarge
12/24/20 00:39:02: Solving for c5n.4xlarge (cpu: 15628.0, memory: 15021.923072)
12/24/20 00:39:02: Reusing solution for a1.metal
12/24/20 00:39:02: Solving for r5d.16xlarge (cpu: 63628.0, memory: 63021.923072)
12/24/20 00:39:02: Reusing solution for m5n.16xlarge
12/24/20 00:39:02: Solving for m5.8xlarge (cpu: 31628.0, memory: 31021.923072)
12/24/20 00:39:02: Reusing solution for i2.8xlarge
12/24/20 00:39:02: Solving for d3.xlarge (cpu: 3628.0, memory: 3021.923072)
12/24/20 00:39:02: Reusing solution for r5ad.xlarge
12/24/20 00:39:02: Solving for g4dn.2xlarge (cpu: 7628.0, memory: 7021.923072)
12/24/20 00:39:02: Reusing solution for m5a.2xlarge
12/24/20 00:39:02: Solving for p4d.24xlarge (cpu: 95628.0, memory: 95021.923072)
12/24/20 00:39:02: Reusing solution for r5n.24xlarge
All solutions found


In [11]:
#solutions[solutions.columns[:-1]]
len(solutions)
#display(solutions[["name", "cpu", "memory", "num_nodes", "cost"]].sort_values(by = "cost",
#                    ascending = "False").reset_index(drop = True))

solutions[["name", "cpu", "memory", "num_nodes", "cost"]].sort_values(by = "cost",
                    ascending = "False").reset_index(drop = True).to_csv("solutions.csv", index=False, )



solutions = solutions[solutions.cost>0].sort_values(by = "cost",
                    ascending = "False").reset_index(drop = True)

In [12]:
best_solution = solutions.iloc[1]

print(f"Best solution: \n")    
print(f"Node: {best_solution['name']}, cpu: {best_solution.cpu},\
          memory: {best_solution.memory}, hourly cost: {best_solution.cost}, number of nodes: {len(best_solution.solution)}")
print("Pod placement:\n")
for i in range(0, len(best_solution.solution)):
    print (f"Node {i}:")
    print (best_solution.solution[i]['pod_name'])
    print (f"Utilization: cpu: {round(best_solution.solution[i].req_cpu_milli_core.sum()/best_solution.cpu*100,2)}%,\
                memory: {round(best_solution.solution[i].req_mem_mb.sum()/best_solution.memory*100, 2)}%")
    

Best solution: 

Node: a1.metal, cpu: 15628.0,          memory: 15021.923072, hourly cost: 3.084, number of nodes: 12
Pod placement:

Node 0:
0               o365-attachment-fetcher-85d8b66996-xrb9k
4               nginx-ingress-controller-78c765749-bnsbt
9                kibana-phishing-kibana-6b8c75f8c4-b4rjm
11                          yara-scanner-9cd796d69-njwt5
14                                konga-69f55f6f65-9s7hd
20                               cerebro-f7fd9554b-cknjp
22                          fetcher-api-57db9bfb6d-h6zh4
24                     grpc-test-client-85c55d84cc-rzwfj
25     malicious-mail-data-stream-1607574709330540699...
30                 cost-analyzer-checks-1607575200-cxhjq
31                             efex-api-665959db5c-dsjs2
42                         impostor-api-666b859458-vzf5n
50                    apollo-mock-server-ffdf4877f-2mnth
54               transaction-enrichment-777dbc47d9-9dkhf
57                   mail-streamer-cron-1607576400-5jvtr
62 